<h1> 아이유팬이 좋아할 만한 다른 아티스트 찾기 <h1>

## 실습 목표
---
- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

## lastfm dataset 가져오기

$ mkdir -p ~/aiffel/EXPLORATION/09/recommendata_iu/data/lastfm-dataset-360K
$ ln -s ~/data/lastfm-dataset-360K/* ~/aiffel/EXPLORATION/09/recommendata_iu/data/lastfm-dataset-360K

## 데이터 준비

$ more ~/aiffel/EXPLORATION/09/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv

Pandas를 활용해 이 데이터를 열어 보기

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/EXPLORATION/09/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 보자.

In [4]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


## 데이터 탐색
---
- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

## 모델 검증을 위한 사용자 초기 정보 세팅

우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주기

In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


## 모델에 활용하기 위한 전처리 (실습)

pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


data의 user_id와 artist 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었으면 훈련을 위한 전처리가 완료

## 사용자의 명시적/암묵적 평가

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용

---
- 한 번이라도 들었으면 선호한다고 판단한다.
- 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## Matrix Factorization(MF)

m명의 사용자들이 n명의 아티스트에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 평가행렬(Rating Matrix)을 만들 것이다.

추천 시스템의 다양한 모델 중 Matrix Factorization(MF, 행렬분해) 모델을 사용

(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어이며, 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해진다. 

하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델

![](https://github.com/MulderKim/EXPLORATION/blob/main/09/etc/E-3v2-2_ekCv9hW.max-800x600.png?raw=true)

(m, k) 사이즈의 Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬


![m=4, n=5, k=2인 MF 모델](https://github.com/MulderKim/EXPLORATION/blob/main/09/etc/E-3v2-3.max-800x600.png?raw=true)

m=4, n=5, k=2인 MF 모델

---
첫 번째 벡터 P<sub>0</sub> = (1, 0.1)은 바로 빨간 모자를 쓴 첫 번째 사용자의 특성(Feature) 벡터

Q 행렬의 첫번째 벡터 Q<sub>0</sub> = (0.9, -0.2)는 해리포터 영화의 특성 벡터

MF 모델은 이 두 벡터를 내적해서 얻어지는 0.88이 바로 R<sub>0,0</sub> 으로 정의되는 사용자의 영화 선호도로 보는 모델

벡터를 잘 만드는 기준은 유저i의 벡터(U<sub>i</sub>)와 아이템j의 벡터(I<sub>j</sub>)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치(M<sub>ij</sub>)와 비슷한지 이며..



U<sub>i</sub> &#183; I<sub>j</sub> = M<sub>ij</sub>


## CSR(Compressed Sparse Row) Matrix

유저 X 아이템 평가행렬은 유저는 36만 명이고 아티스트는 29만 명입으로 행렬로 표현하고 행렬의 각 원소에 정수 한 개 (1byte)가 들어간다면 약 97GB가 필요하며 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능하기 때문에 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix이다

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기

- implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보면 

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 

2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 

3. use_gpu : GPU를 사용할 것인지 

4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝난뒤 2가지 사항을 살펴보면...

1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [21]:
zimin_vector

array([ 0.20901926,  0.60070777,  0.16200316, -0.41259056,  0.2027916 ,
        1.1197585 , -0.7110972 ,  0.60583496,  0.52546895,  0.61950606,
        0.14905703, -0.8690442 ,  0.2654121 , -1.5373814 , -1.3356466 ,
        0.09207828,  0.43072334,  1.1442468 , -0.02451817,  0.05684321,
        0.27796143,  0.21174172,  0.7725191 , -0.20167674,  1.0101323 ,
       -0.57193565, -0.7384177 ,  0.73422456,  0.74019885, -0.67657405,
        0.0435327 , -1.3205544 ,  0.24620798,  0.9046062 ,  0.5841672 ,
       -0.8914771 , -0.52114046,  0.03771257, -1.203079  , -0.34635836,
        0.16804834, -0.867326  ,  0.41654754,  0.47945926,  0.05481883,
        1.5679361 ,  0.86389995,  0.6536913 , -0.3405644 , -0.8829239 ,
        0.6637837 , -0.15878178,  1.081762  , -0.7276846 ,  0.592462  ,
        0.20741072, -0.7606765 , -0.28710538, -1.3288274 , -0.28583115,
       -0.80569434,  0.24468529,  0.37989885,  0.9712091 ,  0.9657601 ,
        0.03149229, -0.8909956 , -0.06948765, -0.88690907,  0.22

In [22]:
black_eyed_peas_vector

array([ 9.31972824e-03,  1.30458418e-02,  1.79365706e-02,  1.48197236e-02,
        1.27608385e-02,  9.59739927e-03,  5.90645988e-03,  1.24716964e-02,
        2.26951130e-02,  6.26348192e-03,  3.67239793e-03,  8.63804296e-03,
        6.93408540e-03, -1.39793092e-02,  3.43707565e-04, -1.14374114e-02,
        1.47572244e-02,  1.67407412e-02,  3.42306495e-03,  1.34130251e-02,
        1.57208031e-03,  1.09662339e-02,  1.68288797e-02,  1.64554995e-02,
        2.87147183e-02, -3.10877338e-03, -2.67837313e-03,  6.68165740e-03,
        9.05041210e-03, -9.21858242e-04,  8.30241852e-03, -2.49872785e-02,
        1.98599920e-02,  1.93425119e-02,  1.51445912e-02, -4.90500219e-03,
        2.75861868e-03,  8.36817361e-03, -1.51973292e-02,  9.31793172e-03,
       -4.56010504e-03,  1.77535918e-02,  6.48722332e-03,  1.75209921e-02,
        3.68739828e-03,  2.10793484e-02,  1.93337798e-02,  1.25218146e-02,
       -8.47573020e-03,  3.30591272e-03,  1.13621084e-02,  6.37303281e-04,
        1.34396916e-02, -

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.5134075

모델이 예시된 queen에 대한 선호도를 어떻게 예측할지 체크

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3070207

## 비슷한 아티스트 찾기 + 유저에게 추천하기

비슷한 아티스트 찾기

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾아볼수 있다.

In [25]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0),
 (277, 0.98584783),
 (28, 0.983517),
 (5, 0.9771919),
 (217, 0.96870023),
 (490, 0.9682188),
 (473, 0.966595),
 (247, 0.965728),
 (418, 0.96102875),
 (910, 0.95119715),
 (268, 0.95020217),
 (694, 0.94814324),
 (1018, 0.9458513),
 (55, 0.94384474),
 (776, 0.9342415)]

(아티스트의 id, 유사도) Tuple 로 반환하고 있음을 확인할 수 있다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 보면..

In [26]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'oasis',
 'placebo',
 'the beatles',
 'u2',
 'nirvana',
 'pink floyd',
 'foo fighters',
 'the smashing pumpkins',
 'arctic monkeys',
 'the cure']

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보면...

In [27]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


2pac 으로 확인해보면...

In [28]:
get_similar_artist('2pac')

['2pac',
 'dr. dre',
 'notorious b.i.g.',
 'nas',
 'the game',
 'jay-z',
 'bone thugs-n-harmony',
 'ice cube',
 'snoop dogg',
 '50 cent']

lady gaga 로 확인해보면...

In [29]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'katy perry',
 'rihanna',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'pink',
 'nelly furtado',
 'kelly clarkson']

유저에게 아티스트 추천하기

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 좋아할 만한 아티스트를 추천받는다. 

filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument 이다.

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.46671638),
 (627, 0.46282348),
 (550, 0.45131847),
 (1800, 0.45056957),
 (2249, 0.44519126),
 (369, 0.44306654),
 (724, 0.4287903),
 (354, 0.42779592),
 (274, 0.4243111),
 (382, 0.4155081),
 (409, 0.41482753),
 (564, 0.4017088),
 (901, 0.40122712),
 (355, 0.3934034),
 (391, 0.38730523),
 (3005, 0.37828487),
 (5555, 0.36662924),
 (618, 0.36296374),
 (621, 0.35721302),
 (24, 0.35548085)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'maroon 5',
 'britney spears',
 'lady gaga',
 'katy perry',
 'justin timberlake',
 'lily allen',
 'nelly furtado',
 'michael jackson',
 'mika',
 'amy winehouse',
 'kanye west',
 'pink',
 'madonna',
 'christina aguilera',
 'duffy',
 'timbaland',
 'the pussycat dolls',
 'alicia keys',
 'jack johnson']

rihanna를 추천해주는 이유는? AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환하한다.

In [33]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.21174005191451922),
 ('black eyed peas', 0.14406602449745617),
 ('jason mraz', 0.0695895732368319),
 ('coldplay', 0.04372703708203175),
 ('maroon5', 0.00025945150381487996)]

filter_already_liked_items=True로 했는데 앞에 maroon 5는 이미 평가한 아이템이 나오는것을 확인할 수 있는데.. 

이것은 implicit 버전 0.4.2에서 생긴 버그이며, 코드는 사람이 만들기 때문에 이렇게 실수할 가능성이 있다.

### 마무리

---

추천시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트를 추천해보았으며 추천시스템 관련 task에서 기본 모델을 만드실 수 있을까??

실습으로 만든 모델은 몇 가지 아쉬운 점이 있는데.... 

1. 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않습니다. 연령대별로 음악 취향이 굉장히 다르지 않을까요? 
2. 유저가 언제 play 했는지에 반영하기 쉽지 않습니다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보세요.

이러한 이유와 딥러닝의 발전으로 MF 이외의 모델 구조도 많이 연구/ 사용되고 있다. 하지만 어떤 추천 모델도 핵심은 MF와 비슷합니다.

유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게(유저에 맞게) 아이템을 보여주거나(Retrieval) 걸러내는(Filtering) 역할입니다.

## 프로젝트 - Movielens 영화 추천 실습

 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보자.
 
 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터로 데이터의 속성 및 특징 그리고 실습방법은 
 
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장.
- 별점 데이터는 대표적인 explicit 데이터이지만 implicit 데이터로 간주하고 테스트해볼 수 있다.
- 별점을 시청횟수로 해석하며 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하고 실습할 예정.

Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 

$ mkdir -p ~/aiffel/EXPLORATION/09/recommendata_iu/data/ml-1m

$ ln -s ~/data/ml-1m/* ~/aiffel/EXPLORATION/09/recommendata_iu/data/ml-1m

### 데이터 준비와 전처리

필요한 라이브러리 가져오기

In [34]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

데이터 불러오기

In [35]:
rating_file_path=os.getenv('HOME') + '/aiffel/EXPLORATION/09/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


rating이 3점 미만 제외...

In [36]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


별점을 시청횟수로 가정하였으므로 rating -> count로

In [37]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [38]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

영화 제목을 보기 위해 메타 데이터 불러오기

In [39]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/EXPLORATION/09/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 분석해 봅시다.

---

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [40]:
print(ratings['movie_id'].nunique())  # ratings에 있는 유니크한 영화 개수

3628


In [41]:
print(ratings['user_id'].nunique())  # rating에 있는 유니크한 사용자 수

6039


In [42]:
# 앞선 두 데이터프레임을 합쳐서 출력해보기
mv_total = pd.merge(ratings, movies, how='left')
mv_total

mv_count = mv_total.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

영화제목 찾기

In [43]:
movies[movies['title'].str.lower().str.contains('star wars')]

,movie_id,title,genre
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi


In [44]:
movies[movies['title'].str.lower().str.contains('x-file')]

,movie_id,title,genre
1840,1909,"X-Files: Fight the Future, The (1998)",Mystery|Sci-Fi|Thriller


In [45]:
movies[movies['title'].str.lower().str.contains('super')]

,movie_id,title,genre
542,546,Super Mario Bros. (1993),Action|Adventure|Children's|Sci-Fi
645,651,"Superweib, Das (1996)",Comedy
850,861,Supercop (1992),Action|Thriller
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
2572,2641,Superman II (1980),Action|Adventure|Sci-Fi
2573,2642,Superman III (1983),Action|Adventure|Sci-Fi
2574,2643,Superman IV: The Quest for Peace (1987),Action|Adventure|Sci-Fi
2838,2907,Superstar (1999),Comedy
3121,3190,Supernova (2000),Adventure|Sci-Fi
3807,3877,Supergirl (1984),Action|Adventure|Fantasy


In [46]:
# 사용자 인덱스 생성
my_id = ratings['user_id'].max() + 1
print(my_id)

6041


In [47]:
ratings.head(10)

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


필요없는 timestamp 칼럼 제거

In [48]:
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [49]:
ratings.tail(10) 

,user_id,movie_id,count
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [50]:
# 내가 좋아하는 영화 데이터 확인
my_mvid = [260, 1196, 1210, 1840, 2640]

my_movie_list = pd.DataFrame(
    {'user_id': [my_id] * 5, 'movie_id': my_mvid, 'count': [5] * 5})

if not ratings.isin({'user_id': [my_id]})['user_id'].any():
    ratings = ratings.append(my_movie_list)
    

ratings.tail(10) 

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,260,5
1,6041,1196,5
2,6041,1210,5
3,6041,1840,5
4,6041,2640,5


user_id와 movie_id 고유한 값으로 만들기

In [60]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,44,5
1,6039,117,5
2,6039,64,5
3,6039,402,5


### CSR matrix를 직접 만들어 봅시다.



In [69]:
n_user = ratings['user_id'].nunique()
n_movie = ratings['movie_id'].nunique()
print(n_user, n_movie)     

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

6040 3628


<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [70]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)
csr_data_transpose = csr_data.T  # als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다.
csr_data_transpose

als_model.fit(csr_data_transpose)  # 모델 훈련

  0%|          | 0/20 [00:00<?, ?it/s]

### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [80]:
# movie_id로 영화 title가져오기
def get_movie_name(idx):
    if idx in movies.movie_id:
        return movies[movies['movie_id'] == idx]['title'].values[0]
    else:
        print('해당 인덱스의 영화가 존재하지 않습니다.')

In [91]:
get_movie_name(1909)

'X-Files: Fight the Future, The (1998)'

In [71]:
# my_id의 벡터
mulder = user_to_idx[my_id]
my_vector = als_model.user_factors[mulder]
my_vector

array([ 0.28167015, -0.4367757 , -0.19584785, -0.13060619, -0.16718808,
       -0.52442765,  0.1866794 , -0.0379503 , -0.2327978 ,  0.06097668,
        0.4868844 ,  0.40792152,  0.5700584 , -0.16114227,  0.37406653,
        0.09831657,  0.4301447 ,  0.03695626, -0.19657406, -0.4427181 ,
        0.18132143, -0.07793729, -0.30657184, -0.16024844, -0.12059655,
       -0.4143333 , -0.4314828 , -0.22450571,  0.27830762,  0.01949837,
        0.29180363,  0.05852951, -0.2878636 ,  0.04060013, -0.05878869,
        0.10618999,  0.31588447,  0.35000315,  0.22829138, -0.06624801,
        0.35907918, -0.42173842,  0.09442679,  0.37316573, -0.15999022,
       -0.32526672, -0.91272014, -0.05173121,  0.09186724,  0.02424346,
       -0.7094922 ,  0.11635602, -0.54433453, -0.20970765, -0.35596797,
       -0.4601389 ,  0.09695107, -0.47788814, -0.4495458 , -0.03223473,
       -0.5954503 , -0.04424626,  0.48913646,  0.5248486 ,  0.38322467,
       -0.42092177, -0.203379  , -0.13185336, -0.23181677,  0.91

In [73]:
# star wars의 벡터
star_wars = movie_to_idx[260]
star_wars_vector = als_model.item_factors[star_wars]
star_wars_vector

array([ 0.01840853, -0.01029846, -0.00502936, -0.01721976, -0.00722847,
       -0.02934353,  0.00781002, -0.00834455,  0.00262773, -0.02014854,
        0.00098833,  0.00631575,  0.02768802,  0.02100278,  0.01424924,
        0.01083007,  0.0090479 ,  0.00014725,  0.0002313 , -0.02147249,
        0.01557962, -0.00603722, -0.00049217, -0.00330826, -0.00190894,
       -0.00503673,  0.01263293, -0.01325885,  0.02800618,  0.00732788,
       -0.00135216, -0.00208999,  0.00738145,  0.02026756, -0.017312  ,
       -0.00334864,  0.01821334,  0.0190076 ,  0.00899172, -0.00705341,
        0.01119415, -0.00418977,  0.01636352,  0.02416321,  0.00553888,
       -0.0110026 ,  0.00258164,  0.02092204, -0.00330288, -0.00543562,
       -0.02372186, -0.01199292, -0.01852865, -0.00344085,  0.01110525,
       -0.02043416,  0.02174855,  0.00854411,  0.01645059,  0.00777862,
       -0.02931215, -0.00563426, -0.0069284 ,  0.00997675,  0.00996804,
       -0.01789904,  0.01499021, -0.01486278,  0.01651676,  0.04

In [93]:
np.dot(my_vector, star_wars_vector)

0.96703196

In [75]:
# x-files의 벡터
xfiles = movie_to_idx[1909]
xfiles_vector = als_model.item_factors[xfiles]
xfiles_vector

array([ 2.67834514e-02, -9.45165567e-03,  2.40545776e-02, -1.10081825e-02,
        2.75757741e-02,  1.07252467e-02, -1.59562659e-02,  1.50151579e-02,
        9.90499929e-03,  2.87578851e-02, -2.96616741e-03, -8.30103178e-04,
       -1.11085428e-02,  1.28132589e-02,  1.08058676e-02,  8.39486439e-03,
        1.03700422e-02,  1.91323534e-02,  1.58813614e-02, -2.92521482e-03,
        1.27099361e-02, -1.03036570e-03,  1.97017076e-03,  1.41014056e-02,
        1.24137765e-02,  7.09117949e-03,  4.37608175e-02,  3.16413827e-02,
        9.43618827e-03, -1.02425450e-02,  2.42013074e-02, -2.20365427e-03,
        6.82744104e-03, -1.90853234e-03, -3.18531878e-03,  2.76460964e-03,
        1.83514208e-02, -7.65355162e-06, -7.10018072e-03,  1.76757500e-02,
        2.68341675e-02,  2.33976599e-02,  2.34048674e-03,  3.04966457e-02,
       -1.15237599e-02,  4.74524731e-03,  7.03363959e-03,  8.28950200e-03,
        2.99495598e-03,  3.36786546e-02, -1.66482907e-02, -1.20291358e-03,
        1.02168415e-02,  

In [94]:
np.dot(my_vector, xfiles_vector)

0.07349926

In [76]:
# super man의 벡터
superman = movie_to_idx[2640]
superman_vector = als_model.item_factors[superman]
superman_vector

array([ 2.28391658e-03, -5.04622096e-03,  1.90308765e-02,  3.23857123e-04,
        5.58507675e-03, -1.00292685e-02,  1.94653887e-02, -1.10329436e-02,
        5.58171375e-03, -2.43950426e-03,  1.84774119e-02,  2.40238458e-02,
       -8.44488852e-04,  9.19145532e-03,  8.74224119e-03,  2.20007244e-02,
        1.31529206e-02,  2.68872385e-03,  5.04800491e-03, -8.87367781e-03,
        2.03083176e-02, -1.76164694e-02,  1.43824397e-02, -1.23937498e-03,
        1.57396554e-03, -1.11797526e-02,  1.56048115e-03, -7.71786086e-03,
        1.06964400e-02,  2.23980029e-03,  9.99818556e-03,  2.05930043e-02,
       -1.50404181e-02,  1.37592526e-02,  2.25955918e-02,  7.61748292e-03,
        2.57235765e-02,  1.30373584e-02,  1.02287577e-02,  2.17577275e-02,
        1.26715256e-02,  2.06770245e-02,  1.10933771e-02,  1.48597630e-02,
        8.88850633e-03, -2.95147635e-02, -2.27677636e-02, -8.52392707e-03,
        2.67410954e-03, -2.13616043e-02, -4.75381181e-04,  6.88395556e-03,
       -3.10917059e-03,  

In [95]:
np.dot(my_vector, superman_vector)

0.63564456

In [97]:
movies[movies['title'].str.lower().str.contains('austin')]

,movie_id,title,genre
1482,1517,Austin Powers: International Man of Mystery (1...,Comedy
2614,2683,Austin Powers: The Spy Who Shagged Me (1999),Comedy


In [98]:
# Austin Powers 의 벡터
austin_powers = movie_to_idx[2683]
austin_powers_vector = als_model.item_factors[austin_powers]
austin_powers_vector

array([-1.03824176e-02,  2.84034275e-02,  1.54515719e-02, -3.99836892e-04,
        1.94320809e-02,  2.57769786e-02,  2.15138588e-02, -2.36170646e-02,
        1.11421095e-02,  1.51458606e-02,  1.03549454e-02,  6.65652426e-03,
       -1.34561630e-03,  4.03153058e-03, -8.41581915e-03,  3.71004194e-02,
       -3.47695779e-03, -1.04378564e-02,  6.63462421e-03, -1.73624028e-02,
        2.46944129e-02, -7.63455266e-03,  1.22000389e-02,  2.26875246e-02,
        1.21204052e-02,  3.72283184e-03,  8.83432291e-03,  7.86652789e-03,
       -2.22086497e-02,  2.20675599e-02,  2.38214340e-03, -1.31031442e-02,
       -3.45029082e-04, -3.63266491e-03,  1.20160752e-03, -3.63965705e-03,
        1.22959362e-02, -1.86731142e-03,  1.74643155e-02,  2.25714371e-02,
        2.87124906e-02, -1.44077260e-02, -2.25452166e-02,  2.18019877e-02,
       -1.92034449e-02,  9.91875771e-03,  1.63120090e-03,  1.01703489e-02,
        2.41313428e-02,  4.05887440e-02,  1.28800208e-02,  1.75546890e-03,
        4.19304520e-03,  

In [99]:
np.dot(my_vector, austin_powers_vector)

-0.029829003

star wars (1977)과 유사한 영화들과의 유사도

In [87]:
movie_id = movie_to_idx[260]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(44, 0.9999999),
 (117, 0.5115858),
 (193, 0.32280773),
 (3134, 0.29173908),
 (1937, 0.29052034),
 (120, 0.2899769),
 (2004, 0.28524235),
 (2731, 0.28400153),
 (3504, 0.28373507),
 (3177, 0.28336096),
 (3460, 0.2820955),
 (3507, 0.28120348),
 (1741, 0.2809558),
 (3052, 0.28084224),
 (3323, 0.2806698)]

X-Files: Fight the Future, The (1998) 과 유사한 영화들과의 유사도

In [89]:
movie_id = movie_to_idx[1909]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(228, 1.0),
 (2736, 0.37140194),
 (3361, 0.36344165),
 (3081, 0.35684922),
 (3302, 0.3563931),
 (2607, 0.3561834),
 (3206, 0.3560002),
 (2883, 0.3533546),
 (3465, 0.35241893),
 (2061, 0.35222447),
 (3196, 0.35214716),
 (2734, 0.3513245),
 (2733, 0.35120356),
 (3458, 0.3511157),
 (2561, 0.35037538)]

Superman (1978) 과 유사한 영화들과의 유사도

In [92]:
movie_id = movie_to_idx[2640]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(526, 1.0000001),
 (763, 0.62248987),
 (1534, 0.4115127),
 (2758, 0.38880783),
 (3025, 0.38605383),
 (2627, 0.38569853),
 (3092, 0.38532284),
 (3057, 0.38485163),
 (3348, 0.38318166),
 (3507, 0.38278568),
 (3210, 0.3826765),
 (2962, 0.38246468),
 (3346, 0.38073623),
 (3532, 0.38038322),
 (3362, 0.3798635)]

### 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [104]:
def get_similar_movie(favorite_movie):
    movie_id = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in similar_movie]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '----------------------------------------------------------------------------------------', sep='\n')

get_similar_movie(260)

     movie_id                                      title  \
257       260  Star Wars: Episode IV - A New Hope (1977)   

                               genre  
257  Action|Adventure|Fantasy|Sci-Fi  
----------------------------------------------------------------------------------------
      movie_id                                              title  \
1178      1196  Star Wars: Episode V - The Empire Strikes Back...   

                                  genre  
1178  Action|Adventure|Drama|Sci-Fi|War  
----------------------------------------------------------------------------------------
      movie_id         title                          genre
1196      1214  Alien (1979)  Action|Horror|Sci-Fi|Thriller
----------------------------------------------------------------------------------------
      movie_id                   title  genre
3511      3580  Up at the Villa (2000)  Drama
----------------------------------------------------------------------------------------
      movi

In [106]:
get_similar_movie(2640)

      movie_id            title                    genre
2571      2640  Superman (1978)  Action|Adventure|Sci-Fi
----------------------------------------------------------------------------------------
      movie_id               title                    genre
2572      2641  Superman II (1980)  Action|Adventure|Sci-Fi
----------------------------------------------------------------------------------------
      movie_id                title                    genre
2573      2642  Superman III (1983)  Action|Adventure|Sci-Fi
----------------------------------------------------------------------------------------
      movie_id                  title          genre
3784      3854  Aimée & Jaguar (1999)  Drama|Romance
----------------------------------------------------------------------------------------
      movie_id                           title          genre
3579      3648  Abominable Snowman, The (1957)  Horror|Sci-Fi
----------------------------------------------------------

In [107]:
get_similar_movie(1909)

      movie_id                                  title                    genre
1840      1909  X-Files: Fight the Future, The (1998)  Mystery|Sci-Fi|Thriller
----------------------------------------------------------------------------------------
      movie_id                   title  genre
1618      1664  Nénette et Boni (1996)  Drama
----------------------------------------------------------------------------------------
      movie_id               title  genre
2138      2207  Jamaica Inn (1939)  Drama
----------------------------------------------------------------------------------------
      movie_id                       title           genre
2768      2837  Bedrooms & Hallways (1998)  Comedy|Romance
----------------------------------------------------------------------------------------
     movie_id                     title        genre
242       245  Glass Shield, The (1994)  Crime|Drama
--------------------------------------------------------------------------------------

### 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [108]:
def recommand_movie(user_id):
    user = user_to_idx[user_id]
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in movie_recommended]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

recommand_movie(my_id)

      movie_id               title                    genre
2572      2641  Superman II (1980)  Action|Adventure|Sci-Fi
--------------------------------------------------------------------
      movie_id                                             title  \
2559      2628  Star Wars: Episode I - The Phantom Menace (1999)   

                                genre  
2559  Action|Adventure|Fantasy|Sci-Fi  
--------------------------------------------------------------------
     movie_id                                              title       genre
740       750  Dr. Strangelove or: How I Learned to Stop Worr...  Sci-Fi|War
--------------------------------------------------------------------
     movie_id          title                         genre
588       592  Batman (1989)  Action|Adventure|Crime|Drama
--------------------------------------------------------------------
     movie_id               title        genre
243       246  Hoop Dreams (1994)  Documentary
---------------------